# Import

In [167]:
path_essentials = 'user_scripts/Timo/from_Simon/script_logic_essentials.py'

path_curdir = 'jupyter/Timo/own/'
import os, time, sys
import numpy as np

# import essentials via run
exec(open(path_essentials).read())
# usual module imports
sys.path.append(path_curdir)

import logging
logging.basicConfig(filename='logfile.log', filemode='w', level=logging.DEBUG)
logger = logging.getLogger(__name__)

memory_dict = OrderedDict()

config_matplotlib_jupyter()
print_welcome_msg()

[Mon Dec 13 16:35:07 2021] Hi, jupyter ready. working dir: C:\Users\Setup3-PC\Desktop\qudi


In [155]:
# check whether fastcounter is setup for gated aqcusition
setup['gated']

True

# Common parameters

In [161]:
# MW parameters
common_rabi_period = 56.0e-9 #M2 149.1e-9#760e-9#245.5e-9 #159.5e-9
common_mw_freq =  2775.0e6#827.64e6 #M2 1503.592e6 #M15, 1507.455e6  #1507.747#7e6  #1503.336e6  #2656.5e6#NV M3 1.497129e9 #NV M2 1.503316e9 
common_mw_amp = 0.25

# Readout
# RELOAD logic_essentials and upload new seq when changing!
common_t_laser_pulsed = 3e-6   # for pulsed mes (non NI acq)

# Ramsey
common_mw_ramsey_offset = 1000e3   # Hz, detuning, 
common_mw_ramsey_off_from_calib = 1340e3 #0.1e6 #[0.66e6, 0.72e6]   # Hz, detuning,taken for checking valid




# Library

## Basics

In [152]:
import pickle
from scipy.optimize import curve_fit
from scipy.stats import poisson

def setup_podmr_params(f_res, p_scale=10, p_scale_onlyperiod=1, f_step=0.1e6, n_f=50, t_pi_manual=None,
                      t_wait_2=0.0e-9):
    # @param p_scale_onlyperiod: scales rabi period according to given power factor.
    #                            useful, if externally controlling poewr
    qmeas=OrderedDict()
    qmeas['name'] = 'pulsedodmr'
    setup['laser_length'] = common_t_laser_pulsed
    
    if t_pi_manual:
        qmeas['rabi_period'] = 2*t_pi_manual
    else:
        qmeas['rabi_period'] = common_rabi_period*p_scale*p_scale_onlyperiod
    qmeas['freq_step'] = f_step
    qmeas['num_of_points'] = n_f
    qmeas['freq_start'] = f_res - (qmeas['freq_step'] * qmeas['num_of_points'])/2.
    if p_scale != 1:
        qmeas['microwave_amplitude'] = common_mw_amp/(2.*p_scale)
    else:
        qmeas['microwave_amplitude'] = common_mw_amp
    qmeas['measurement_time'] = 1500 
    qmeas['optimize_time'] = None
    qmeas['fit_experiment'] = 'lor_dip'
    
    return qmeas


def setup_rabi_params(tau_step=None, p_scale=1):
    
    qmeas=OrderedDict()
    qmeas['name'] = 'Rabi'
    setup['laser_length'] = common_t_laser_pulsed
    
    if tau_step is None:   
        tau_step = 5e-9
    qmeas['tau_step'] = tau_step
    qmeas['tau_start'] = 0e-9
    qmeas['num_of_points'] = 50
    qmeas['microwave_frequency'] = common_mw_freq #+ 0.5*3.03e6
    if p_scale != 1:
        qmeas['microwave_amplitude'] = common_mw_amp/(2.*p_scale)
    else:
        qmeas['microwave_amplitude'] = common_mw_amp
    qmeas['measurement_time'] = None
    qmeas['optimize_time'] = None
    qmeas['freq_optimize_time'] = None
    #qmeas['fit_experiment'] = None  #'cosine'
    #qmeas['analysis_interval'] = 3
    #qmeas['n_sweeps'] = 5
    
    return qmeas

def setup_ramsey_params(tau_start=100e-9, n_tau=1, tau_step=None):
    qmeas=OrderedDict()
    qmeas['name'] = 'Ramsey'
    setup['laser_length'] = common_t_laser_pulsed
    
    qmeas['rabi_period'] = common_rabi_period
    qmeas['microwave_frequency'] = common_mw_freq + common_mw_ramsey_offset
    qmeas['microwave_amplitude'] = common_mw_amp
    qmeas['tau_start'] = tau_start
    if tau_step is None:
        qmeas['tau_step'] = 50e-9
    else:
        qmeas['tau_step'] = tau_step
    qmeas['num_of_points'] = n_tau
    qmeas['alternating'] = False
    
    qmeas['from_list'] = False
    
    qmeas['optimize_time'] = None
    #qmeas['freq_optimization_time'] = None
    #qmeas['analysis_interval'] = 3
    if n_tau > 1:
        qmeas['fit_experiment'] = 'sine_decay'
 
    return qmeas

def setup_xy8_tausweep_params(tau_start=100e-9, n_tau=20, tau_step=None, xy8_order=1, alternating=False):
    qmeas=OrderedDict()
    qmeas['name'] = 'xy8_tau'
    setup['laser_length'] = common_t_laser_pulsed
    
    qmeas['rabi_period'] = common_rabi_period
    qmeas['microwave_frequency'] = common_mw_freq 
    qmeas['microwave_amplitude'] = common_mw_amp
    
    qmeas['xy8_order'] = xy8_order
    qmeas['tau_start'] = tau_start
    if tau_step is None:
        qmeas['tau_step'] = 100e-9
    else:
        qmeas['tau_step'] = tau_step
        
    
    qmeas['num_of_points'] = n_tau
    qmeas['alternating'] = alternating
    
    qmeas['from_list'] = False
    
    qmeas['optimize_time'] = None
    #qmeas['freq_optimization_time'] = None
    #qmeas['analysis_interval'] = 3
    if n_tau > 1:
        qmeas['fit_experiment'] = 'Lorentz dip'
 
    return qmeas

def setup_xy8_nsweep_params(tau=500e-9, n_n=20, xy8_start=1, xy8_step=1, alternating=False):
    qmeas=OrderedDict()
    qmeas['name'] = 'xy8_nsweep'
    setup['laser_length'] = common_t_laser_pulsed
    
    qmeas['rabi_period'] = common_rabi_period
    qmeas['microwave_frequency'] = common_mw_freq 
    qmeas['microwave_amplitude'] = common_mw_amp
    
    qmeas['tau'] = tau
    qmeas['xy8_start'] = xy8_start
    qmeas['xy8_step'] = xy8_step

        
    
    qmeas['num_of_points'] = n_n
    qmeas['alternating'] = alternating
    
    qmeas['from_list'] = False
    
    qmeas['optimize_time'] = None
    #qmeas['freq_optimization_time'] = None
    #qmeas['analysis_interval'] = 3
    if n_n > 1:
        qmeas['fit_experiment'] = 'sine_decay'
 
    return qmeas

def run_pdomr(mes_params, save_tag=None, generate_new=True):
    
    up_to = (mes_params['freq_start'] + mes_params['freq_step'] * mes_params['num_of_points'])
    laser_off()
    if save_tag is None:
        save_tag='pOdmr_{:.2f}_{:.2f}MHz'.format(mes_params['freq_start']/1e6, up_to/1e6)
    do_experiment(experiment='podmr_pentacene', qm_dict=mes_params, meas_type=conventional_measurement, meas_info=add_conventional_information, 
                  generate_new=generate_new, save_tag=save_tag, load_tag='')

def run_generic(mes_params, save_tag=None, generate_new=True):
    laser_off()
    if save_tag is None:
        save_tag=f"{mes_params['name']}"
    mes_params['savetag'] = save_tag
    do_experiment(experiment=mes_params['experiment'], qm_dict=mes_params, meas_type=conventional_measurement,
                  meas_info=add_conventional_information,
                  generate_new=generate_new, save_tag=save_tag, load_tag='')

def run_generic_autoexp(qmeas_list, generate_new=False, optimize_between_exps=False, poi_list=None):
        
    autoexp = OrderedDict()
    automeas = OrderedDict()

    automeas['list_pois'] = poi_list if poi_list else [""]
    automeas['generate_new'] = generate_new
    automeas['optimize_between_experiments'] = optimize_between_exps

    for qmeas in qmeas_list:
        exp_name = qmeas['name']
        qmeas['type'] = qmeas['experiment']
        qmeas['meas_type'] = conventional_measurement
        qmeas['meas_info'] = add_conventional_information

        autoexp[exp_name] = copy.deepcopy(qmeas)
    
    do_automized_measurements(qm_dict=automeas, autoexp=autoexp)
    

def create_temp_dir():
    #qudi_dir = os.getcwd()
    
    folder_dir = 'temp'
    try:
        os.mkdir(folder_dir)
    except FileExistsError as e:
        pass   # already exists
    
    return folder_dir 
    
def get_counts(t_avg_s=0.1):
    
    # stop refocusing for getting counts
    refocus_running = True
    try:
        if poimanagerlogic.time_until_refocus == -1:
            refocus_running = False
        else:
            poimanagerlogic.toggle_periodic_refocus(False)
    except NameError:
        refocus_running = False
        
    
    f_count = counterlogic.get_count_frequency()
    n_samples = int(t_avg_s * f_count)
    n_samples_max = counterlogic.get_count_length()
    if n_samples > n_samples_max:
        n_samples = n_samples_max
        
    # wait until refocus done for sure
    if refocus_running:
        timeout = False
        timeout_t_s = 5
        t_start = time.time()

        while(time.time() - poimanagerlogic._last_refocus < t_avg_s and not timeout):
            time.sleep(0.1)
            if time.time() - t_start > timeout_t_s:
                timeout = True
                
    cts = np.median(counterlogic.countdata[0][-n_samples:])    # counts per s
    
    if refocus_running:
        poimanagerlogic.toggle_periodic_refocus(True)
    
    return cts
        


# Laser options

## Optimize position

In [ ]:
laser_on()
optimize_position()

## Laser On

In [17]:
#laser_on(pulser_on = False)
laser_on()
counterlogic.startCount()

0

## Laser Off

In [20]:
laser_off(pulser_on=False)
counterlogic.stopCount()


# Frequency Optimization parameters

In [ ]:
optimize_freq_dict=OrderedDict()
optimize_freq_dict['mw_optimization_method'] = 'pulsedodmr'
optimize_freq_dict['optimize_generate_new'] = False
optimize_freq_dict['save_tag'] = 'optimize_frequency'
optimize_freq_dict['optimize_fit_method'] = 'N15'
optimize_freq_dict['parameters2update'] = {'center': 'microwave_frequency'}

optimize_freq_dict['name'] = 'Pulsed_ODMR'
optimize_freq_dict['rabi_period'] = 1800e-9
optimize_freq_dict['freq_start'] = 4.127e9
optimize_freq_dict['freq_step'] = 0.2e6
optimize_freq_dict['num_of_points'] = 50
optimize_freq_dict['microwave_amplitude'] = common_mw_amp/15
optimize_freq_dict['meas_type'] = conventional_measurement
optimize_freq_dict['meas_info'] = add_conventional_information

optimize_freq_dict['measurement_time'] = 30
optimize_freq_dict['optimize_time'] = None
optimize_freq_dict['optimize_generate_new'] = True

#  Preperation mes

## Pulsed ODMR

In [159]:
p_scale_manual = 1
uglobals.abort.clear()

if p_scale_manual != 1:
    logger.warning("Manually reduce AWG ampl / AMP gain by a factor of {} / {}!".format(2*p_scale_manual, np.sqrt(2*p_scale_manual)))
#qmeas = setup_podmr_params(f_res=common_mw_freq, p_scale=50, p_scale_onlyperiod=p_scale_manual, f_step=20e3) # range ~ 1 MHz

#qmeas = setup_podmr_params(f_res=common_mw_freq, p_scale=1, f_step=3000e3, n_f=150)     # wide band, ~ 300 MHz
#qmeas = setup_podmr_params(f_res=common_mw_freq, p_scale=1, f_step=1000e3, n_f=50)     # wide band, ~ 2.5 MHz
qmeas = setup_podmr_params(f_res=common_mw_freq, p_scale=1, f_step=1000e3, n_f=50, t_wait_2=50e-6)     
#qmeas = setup_podmr_params(f_res=common_mw_freq, p_scale=5, f_step=500e3, n_f=300)     # wide band, ~ 10 MHz
#qmeas = setup_podmr_params(f_res=common_mw_freq, p_scale=5, f_step=500e3, n_f=75)
#qmeas = setup_podmr_params(f_res=common_mw_freq, p_scale=10, f_step=50e3, n_f=75) # range ~ 5 MHz
#qmeas = setup_podmr_params(f_res=common_mw_freq, p_scale=20, f_step=25e3, n_f=75, t_pi_manual=8e-6)
#qmeas = setup_podmr_params(f_res=common_mw_freq, p_scale=30, n_f=75)
#qmeas = setup_podmr_params(f_res=common_mw_freq, p_scale=40, f_step=40e3, n_f=450) # range ~ 3*6 MHz
#qmeas = setup_podmr_params(f_res=common_mw_freq, p_scale=60, f_step=20e3, n_f=100) # range ~ 1 MHz, lowest AWG power

run_generic(qmeas)

## Rabi

In [37]:
uglobals.abort.clear()
qmeas = setup_rabi_params(tau_step=20e-9)   # 10e-9
#qmeas = setup_rabi_params(tau_step=250e-9, p_scale=20)
run_generic(qmeas)

## Ramsey


In [43]:
tau_start = 0.5e-6
n_sweeps = 0#1e3

uglobals.abort.clear()
#qmeas = setup_ramsey_params(tau_start, n_tau=100, tau_step=100e-9)     
qmeas = setup_ramsey_params(tau_start, n_tau=300, tau_step=100e-9)
#qmeas = setup_ramsey_params(tau_start, n_tau=200, tau_step=250e-9)      # 75ns: res like mfl
#qmeas = setup_ramsey_params(tau_start, n_tau=400, tau_step=50e-9)     # high res    
#qmeas = setup_ramsey_params(tau_start, n_tau=400, tau_step=100e-9)     # high res  
x, z = run_generic(qmeas, n_sweeps_stop=n_sweeps)
z_binary = 0 #majority_vote(z[0], z_thres=common_mfl_z_thres)

logger.info("Ramsey with offset {} MHz, {} sweeps: {} -> {}".format(common_mw_ramsey_offset/1e6, n_sweeps, z, z_binary))


## XY 8 

tau sweep

In [43]:
#tau_start = 0.2e-6
tau_start = 0.415e-6

uglobals.abort.clear()
# narrow resonance NV M17
qmeas = setup_xy8_tausweep_params(tau_start, n_tau=140, tau_step=0.5e-9, xy8_order=22, alternating=True)

#qmeas = setup_xy8_tausweep_params(tau_start, n_tau=10, tau_step=25e-9, xy8_order=800, alternating=True)


# larger sweeps
#qmeas = setup_xy8_tausweep_params(tau_start, n_tau=75, tau_step=10e-9, xy8_order=4, alternating=True) 
#qmeas = setup_xy8_tausweep_params(tau_start, n_tau=75, tau_step=3e-9, xy8_order=4, alternating=True) 
#qmeas = setup_xy8_tausweep_params(tau_start, n_tau=250, tau_step=20e-9, xy8_order=4, alternating=True)
#qmeas = setup_xy8_tausweep_params(tau_start, n_tau=75, tau_step=5e-9, xy8_order=8, alternating=True) 

run_generic(qmeas)

n sweep

In [21]:
tau = 300e-9#440.0e-9
tau = 441.6e-9

uglobals.abort.clear()
#qmeas = setup_xy8_nsweep_params(tau, n_n=25, xy8_start=4, xy8_step=1, alternating=True)
qmeas = setup_xy8_nsweep_params(tau, n_n=15, xy8_start=4, xy8_step=100, alternating=True)  # long t_evol, mem limit
#qmeas = setup_xy8_nsweep_params(tau, n_n=25, xy8_start=1, xy8_step=1, alternating=True)  

run_generic(qmeas)

In [ ]:
a few xy8 tau sweeps

In [54]:
uglobals.abort.clear()

tau_start = 0.5e-6
xy_orders = np.arange(5, 16, 1)

for i, o in enumerate(xy_orders):
    if uglobals.abort.is_set():
        break
    try:
        #pass
        #"""
        laser_on()
        time.sleep(1)
        optimize_position()
        optimize_position()
        laser_off()
        #"""
    except Exception as e:
        logger.warning("auto XY8 run {}, skipping failed tracking: {}".format(i, str(e)))
    if uglobals.abort.is_set():
        break
        
    qmeas = setup_xy8_tausweep_params(tau_start, n_tau=50, tau_step=20e-9, xy8_order=o, alternating=True)     
    qmeas['measurement_time'] = 180
    run_generic(qmeas)

# Auto pilot multiple NVs

In [164]:
# In order to stop the automized measurement insert pulsedmasterlogic.break_variable = True into manager
import copy
uglobals.abort.clear()

# for pois=None, all experiment will run on the current confocal pos
pois = None
# pois = poimanagerlogic.poi_names[:]
logger.info("Autopilot is taking of for pois {}".format(pois))


qmeas_list = []

qexp = setup_podmr_params(common_mw_freq)
qexp['experiment'] = 'pulsedodmr'
qexp['name'] = 'podmr'


qexp['laser_length'] = common_t_laser_pulsed

qexp['bin_width'] = 128e-9
qexp['wait_time'] = 0.0e-9

qexp['rabi_period'] = common_rabi_period#*4
qexp['freq_start'] = common_mw_freq - 3e6
qexp['freq_step'] = 1e6
qexp['num_of_points'] = 30
qexp['microwave_amplitude'] = common_mw_amp#/4
qexp['measurement_time'] = 20
qexp['optimize_time'] = None
qexp['freq_optimize_time'] = None
qexp['generate_new'] = False
qexp['meas_type'] = conventional_measurement
qexp['meas_info'] = add_conventional_information
# if empty  or commented out, no fit is used
qexp['fit_experiment'] = 'lor_dip'
qexp['fit_parameter'] = 'Position'
qexp['update_parameters'] = {'Rabi': 'microwave_frequency', 'xy8': 'microwave_frequency', 'xy8_2': 'microwave_frequency'}

qmeas_list.append(copy.deepcopy(qexp))
#"""

#Experiment2
#"""
qexp = setup_rabi_params(tau_step=10e-9)
qexp['experiment'] = 'rabi'
qexp['measurement_time'] = 30
qexp['optimize_time'] = None
qexp['freq_optimize_time'] = None
qexp['generate_new'] = True
qexp['meas_type'] = conventional_measurement
qexp['meas_info'] = add_conventional_information
# if empty  or commented out, no fit is used
qexp['fit_experiment'] = 'sine'
qexp['fit_parameter'] = 'Period'
qexp['update_parameters'] = {'ramsey': 'rabi_period'}  # feed forware parameters
qmeas_list.append(copy.deepcopy(qexp))

"""
#Experiment3
qexp = setup_ramsey_params(tau_start=50e-9, n_tau=400, tau_step=50e-9)  # res= 50 kHz 
qexp['experiment'] = 'ramsey'
qexp['measurement_time'] = 120
qexp['generate_new'] = True
qexp['meas_type'] = conventional_measurement
qexp['meas_info'] = add_conventional_information
qexp['fit_experiment'] = 'double_sine_decay'
qmeas_list.append(copy.deepcopy(qexp))

"""

laser_off()

try:
    run_generic_autoexp(qmeas_list,  generate_new=True, optimize_between_exps=False, poi_list=pois)
except Exception as e:
    stack_trace = traceback.format_exc()
    logger.exception("auto run failed: {}.\n".format(stack_trace))




In [83]:
traceback

<module 'traceback' from 'C:\\ProgramData\\Anaconda3\\envs\\qudi\\lib\\traceback.py'>